# Training data

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# imports
# feature selection
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_selection import RFECV
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import VarianceThreshold
from sklearn.svm import LinearSVC
from sklearn.svm import SVR

# models
from sklearn.pipeline import Pipeline
from sklearn import svm
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB

# scoring
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score

np.random.seed(0)

df = pd.read_csv('npf_train.csv')

class2 = np.array(["event"]*df.shape[0], dtype="object")
class2[df["class4"]=="nonevent"] = "nonevent"
df["class2"] = class2
df["class2"] = df["class2"].astype("category")

not_needed_columns = ["partlybad", "id", "date"]
df = df.drop(not_needed_columns, axis=1)

class2_categories = {
    "nonevent": 0,
    "event": 1
}

class4_categories = {
    "nonevent": 0,
    "Ia": 1,
    "Ib": 2,
    "II": 3
}

def categorize_class2(row):
    row["class2"] = class2_categories[row["class2"]]
    
    return row
def categorize_class4(row):
    row["class4"] = class4_categories[row["class4"]]
    
    return row

df = df.apply(lambda row: categorize_class2(row), axis=1)
df = df.apply(lambda row: categorize_class4(row), axis=1)
df["class2"] = df["class2"].astype("category")
df["class4"] = df["class4"].astype("category")

df = df.sample(frac=1).reset_index(drop=True)

# Training

In [3]:
# Variables for results
initial_result = {}
result_df = pd.DataFrame(data=initial_result)

# Parameters
y_col = "class4"
cv_features = StratifiedKFold(10)
cv = 10
step_features = 1

# Data
X_cols = list(filter(lambda col: col != y_col and col != "class2", df.columns))
X_train = df[X_cols]
y_train = df[y_col]

# Train
clf = RandomForestClassifier(random_state=0)
estimator = RandomForestClassifier(random_state=0)
pipe = Pipeline(
    [('scaler', StandardScaler()), 
    ("feature_selection", RFECV(estimator=estimator, step=step_features, cv=cv_features, scoring="accuracy")), 
    ("clf", clf)]
)

scores = cross_val_score(pipe, X_train, y_train, cv=cv)
cv_score = scores.mean()

# Fitted model
pipe.fit(X_train, y_train)

# Prediction

In [ ]:
chosen_features =  list(X_train.columns[pipe.steps[1][1].support_]), 

# Read and clean the test data
df = pd.read_csv('npf_test_hidden.csv')

class2 = np.array(["event"]*df.shape[0],dtype="object")
class2[df["class4"]=="nonevent"] = "nonevent"
df["class2"] = class2
df["class2"] = df["class2"].astype("category")

not_needed_columns = ["partlybad", "id", "date", "class4"]
df = df.drop(not_needed_columns, axis=1)

X_df = df.loc[:, ~df.columns.isin(['class2', 'class4'])].copy()
X = X_df.values

pred = pipe.predict_proba(X)


# Results

In [ ]:
# p column
p_nonevent = pred[:,0]
p_event = np.sum(pred[:,1:], axis=1)
np.stack((p_nonevent, p_event))

results = pd.DataFrame(data={"nonevent":pred[:,0], "Ia":pred[:,1], "Ib":pred[:,2], "II":pred[:,3], "prediction":np.argmax(pred, axis=1), "p":p_event})
results

In [ ]:
# accuracy
cv_score